In [1]:
import copy
import helpers, dirs
import langid
from genderizer.genderizer import Genderizer

In [28]:
discovery_list, discovery_header = helpers.load_csv(dirs.dirs_dict["discoveries"]["instagram"])

In [29]:
len(discovery_list)

41065

In [7]:
for i,profile in enumerate(discovery_list):
    discovery_list[i]['follow_ratio'] = profile['num_followers']/profile['num_follows'] if profile['num_follows'] > 0 else True

In [3]:
sum(p['num_follows'] * int(p['degree']==1) for p in discovery_list)

4780941

In [11]:
def is_english(description):
    try:
        lang, prob = langid.classify(description)
    except:
        return False
    return 'en' == lang

In [12]:
def is_male(name, description):
    # if they don't have a name listed,
    # then they can't be influencers
    if not name:
        return False
    first_name = name.split(' ')[0]
    try:
        gender = Genderizer.detect(firstName = first_name, text=description)
    except:
        # if it doesn't know what gender
        # the person is, we'll still keep them
        return True
    return 'male' == gender

In [13]:
def filter_list(list_of_dicts, key, value, gte=True):
    return filter(lambda profile: bool(profile[key] >= value) == gte, list_of_dicts)    

In [14]:
def filter_function(list_of_dicts, fn, key_args):
    return filter(lambda profile: fn(*[profile[k] for k in key_args]), list_of_dicts)

In [22]:
filters = (
            # less than filters
            ('num_followers', 1000000, False),
    
            # greater than filters
            ('num_followers', 10000,   True),
            ('num_posts',     100,     True),
            ('follow_ratio',  10,      True),
          )

In [23]:
new_list = copy.deepcopy(discovery_list)
for key, value, gte in filters:
    new_list = filter_list(new_list, key, value, gte)
new_list = filter_function(new_list, is_english, ['description'])

In [25]:
new_list = filter_function(new_list, is_male, ['name', 'description',])

In [27]:
helpers.write_csv(dirs.dirs_dict["discoveries"]["instagram"]+'-filtered', new_list, discovery_header, sort=['num_followers',], sort_rev=True)